In [4]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForMaskedLM, LineByLineTextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import re

# Загрузка модели BERT и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Чтение файлов
with open('/content/train.txt', 'r', encoding='utf-8') as f:
    train_text = f.read()

with open('/content/test.txt', 'r', encoding='utf-8') as f:
    test_text = f.read()

# Сохраняем train.txt в файл для дообучения
with open('/content/train_for_bert.txt', 'w', encoding='utf-8') as f:
    f.write(train_text)

# Создаем датасет и коллатор для дообучения
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='/content/train_for_bert.txt',
    block_size=128,
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/tran

In [5]:

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)



In [7]:
# Настройка обучения
training_args = TrainingArguments(
    output_dir='/kaggle/working/bert_output',
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

# Инициализация тренера
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Дообучение модели
trainer.train()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: daniel-teplov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,2.423200
1000,1.983800


TrainOutput(global_step=1160, training_loss=2.1587507050612875, metrics={'train_runtime': 244.9094, 'train_samples_per_second': 75.742, 'train_steps_per_second': 4.736, 'total_flos': 240902317490400.0, 'train_loss': 2.1587507050612875, 'epoch': 5.0})

In [18]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [16]:
device = 'cuda'

In [17]:
model = model.to(device)

In [11]:
mask_pattern = r'\*\[(\d+)\]\*'

results = {}


mask_matches = list(re.finditer(mask_pattern, test_text))

In [21]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForMaskedLM
import re

# Определяем устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Загрузка модели BERT и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model = model.to(device)
model.eval()

# Чтение файлов
with open('/content/train.txt', 'r', encoding='utf-8') as f:
    train_text = f.read()

with open('/content/test.txt', 'r', encoding='utf-8') as f:
    test_text = f.read()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:

mask_pattern = r'\*\[(\d+)\]\*'

results = {}


mask_matches = list(re.finditer(mask_pattern, test_text))

for match in mask_matches:
    mask_id = int(match.group(1))

    start_pos = max(0, match.start() - 200)
    end_pos = min(len(test_text), match.end() + 200)

    context_before = test_text[start_pos:match.start()]
    context_after = test_text[match.end():end_pos]
    masked_text = context_before + tokenizer.mask_token + context_after

    # Токенизация
    inputs = tokenizer(masked_text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    mask_token_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0]

    if len(mask_token_index) > 0:
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        mask_token_logits = logits[0, mask_token_index, :]

        top_token_ids = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

        for token_id in top_token_ids:
            predicted_token = tokenizer.convert_ids_to_tokens([token_id])[0]

            if predicted_token.startswith('##'):
                predicted_token = predicted_token[2:]

            if predicted_token.lower() in train_text.lower():
                results[mask_id] = predicted_token.lower()
                break

        if mask_id not in results:
            results[mask_id] = tokenizer.convert_ids_to_tokens([top_token_ids[0]])[0].lower()
    else:
        masked_text = test_text.replace(match.group(), tokenizer.mask_token)
        inputs = tokenizer(masked_text, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        mask_token_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0]

        if len(mask_token_index) > 0:
            with torch.no_grad():
                outputs = model(**inputs)

            logits = outputs.logits
            mask_token_logits = logits[0, mask_token_index, :]
            top_token_id = torch.argmax(mask_token_logits, dim=1).item()
            predicted_token = tokenizer.convert_ids_to_tokens([top_token_id])[0]

            if predicted_token.startswith('##'):
                predicted_token = predicted_token[2:]

            results[mask_id] = predicted_token.lower()
        else:
            results[mask_id] = "the"



In [23]:
submit_data = [{"word_id": k, "word": v} for k, v in sorted(results.items())]
submit_df = pd.DataFrame(submit_data)
submit_df.to_csv('submit.csv', index=False)